In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Label

Labels of enhancers remain strongly unbalance, even with dropping. Almost all values of enhancers labels are 0 (third quartile is 0).

I decided to set promoters threshold equal to 1 and don't drop values between 0 and 1 because this improve balance between classes but not enough to eliminate samples from the dataset. 
About enhancers the threshold is set to 0 because this leads classes to be as balanced as possible.

I'm not focusing on the biological meaning of my choices, the threshold are chosen to obtain a dataset useful for learning. 

In [ ]:
promoters_labels = pd.read_csv('./pre_processed_dataset/labels_promoters.csv', index_col = [0,1,2,3])
enhancers_labels = pd.read_csv('./pre_processed_dataset/labels_enhancers.csv', index_col = [0,1,2,3])
promoters = pd.read_csv('./pre_processed_dataset/epigenomes_promoters.csv', index_col = [0,1,2,3])
enhancers = pd.read_csv('./pre_processed_dataset/epigenomes_enhancers.csv', index_col = [0,1,2,3])

epigenomes = {
    "promoters": promoters,
    "enhancers": enhancers
}

labels = {
    "promoters": promoters_labels,
    "enhancers": enhancers_labels
}